# Quantifying GCQuant for GCMS & GC-TIC



To quantify GC data with GCQuant you will need three things:

1. A dataframe containing library_ids with corresponding retention times
2. A dataframe containing area with corresponding retention times
3. A dataframe with calibration curve information.

After reading in GCMS data, concentrations can be determined using gcquant. gcquant has the following hierarchy:

- no one
- no two
- no three

## LibraryIDs and Areas
For this example we will use the GC - Agilent class to create the first two items. This is easily done with the code below. For a full description of the class look [here](https://github.com/blakeboswell/chemtbd/blob/master/example.ipynb).

In [ ]:
from chemtbd.io import Agilent
agi = Agilent.from_root('data/test3')

lib = agi.results_lib
area = agi.results_tic

In [ ]:
lib.head()

In [ ]:
area.head()

## Create Calibration DataFrame

Great! Now that the species have a corresponding area, the next step is to create calibration curves. To create calibration curves the following 3 things will be needed, and this information will need to be captured in a dataframe.

- identify files within the subfolder that contain calibration curve data
- provide the known concentrations of each species in each file

We recommend either putting the data into a csv file and importing it using pandas or creating a pandas dataframe. Below is how this can be performed. 

#### IMPORTANT: 
1. The dataframe headers must have a library_id column and the remaining columns should be the file names for each standard. 
2. Be sure the file names are exactly the same as the files in the subfolder including any extention in the name (e.g. ".D").
3. UNITS UNITS UNITS. The concentrations should be entered in molar (mol/L).

In [ ]:
import pandas as pd
standards = pd.read_csv('data/standards.csv')
standards.head()

## Analysis with GCQuant

Now that we have everything we need, lets load the data into GCQuant and see what analysis is now available.

In [ ]:
from chemtbd.io import GCQuant
gcq = GCQuant(lib,area,standards)

### Getting compiled data
GCQuant compiled both the `lib` and the `area` dataframes which you provided to the class into one dataframe. The rows were matched based on the squared difference of the retention time from each row (i.e. each peak). As shown below this compiled dataframe is now available.

In [ ]:
gcq.compiled.head()

### Calibration Curve Data
The calibration curve data is now available as well. The class provides statistics as well as easily viable plots to make sure your curves area linear.

In [ ]:
gcq.stdcurves.head()

It is good to make sure all of the correlations have a high r<sup>2</sup> for the linear fit. We can use `.sort('rvalue')` to order the dataframe by r<sup>2</sup> value from lowest to highest.

In [ ]:
gcq.stdcurves.sort('rvalue').head()

### Quantifying Concentrations

Now that we know our calibration curves have a good linear fit, we can easily calculate the unknown concentrations of each species using GCQuants `.concentrations` property.

In [ ]:
gcq.concentrations.head()

Nice! We now have the calculated concentrations in the `conc` column.  Addtionally GCQuant has provided the normalized concentration, `conc%`, and normalized area, `area%`.

But the dataframe isn't always the best way to view the results. GCQuant has a few built in reporting methods to easily "see" your results

## Report Results
### Pivot Table 
Pivot tables are generally a nice way of organizing printing data. The `.pivot` method is relatively simple to implement on your own. If there are other cases which could be useful below is an example of creating our own pivot table with pandas `.pivot_table` method.

*Note: it is important to reset the index of the gcq.concentrations dataframe so the `key` is then a column and not the index.*

In [ ]:
pd.pivot_table(gcq.concentrations.reset_index(),
               index='library_id',
               columns='key',
               values='conc').head()

We can also just look at our experimental data without the standard curve data by simply calling `gcq.concentrations_exp`. Similarly, we can get just the standard curve data `gcq.concentrations_std` and place either in a pivot table.

In [ ]:
pd.pivot_table(gcq.concentrations_exp.reset_index(),
               index='library_id',
               columns='key',
               values='conc').head()

In [ ]:
pd.pivot_table(gcq.concentrations_std.reset_index(),
               index='library_id',
               columns='key',
               values='conc').head()

### Bar Plots

Your data can easily be plotted with GCQuant. There are many plotting packages available in python, but here Bokeh is used to prepare a few simple examples which you can use as a quick check or to build upon.

Bokeh has very good documentation so detailed explanation of the code can be understood by referencing Bokeh's documentation.

First import the required classes

In [ ]:
from bokeh.charts import Bar, show, output_notebook
from bokeh.layouts import row

To see the charts displayed in the notebook use the line below.

In [ ]:
output_notebook()

Now a bar chart of the concentration percents can be created

In [ ]:
bar_conc_per = Bar(gcq.concentrations_exp.reset_index(),
                       label ='key',
                       values = 'conc%',
                       agg='sum',
                       stack='library_id',
                       title = 'Concentration Percentage')
show(bar_conc_per)

To plot absolute values we can create a similiar plot.

In [ ]:
bar_conc_abs = Bar(gcq.concentrations_exp.reset_index(),
                       label ='key',
                       values = 'conc',
                       agg='sum',
                       stack='library_id',
                       title = 'Concentration')
show(bar_conc_abs)

The same data but not stacked.

In [ ]:
bar_conc_abs_nostack = Bar(gcq.concentrations_exp.reset_index(),
                       label ='key',
                       values = 'conc',
                       group = 'library_id',
                       title = 'Concentration')
show(bar_conc_abs_nostack)

Combining plots

In [ ]:
show(row(bar_conc_per,bar_conc_abs_nostack)) 